In [39]:
using Statistics
using LinearAlgebra
using Random
using Plots
using LaTeXStrings
using TensorCrossInterpolation
# import TensorCrossInterpolation: TensorCI, CachedFunction
import TensorCrossInterpolation as TCI
# using qtt_option
# import qtt_option as qo
# using TCIITensorConversion
# using ITensors

In [40]:
# struct to count number of function calls and sampling points

mutable struct WrapFunc{F}
    f::F
    count::Int64
end

function (w::WrapFunc)(z)
    w.count += 1
    return w.f(z)
end

In [41]:
struct GenNDfunc{F} <: Function
    d::Int
    func::F
    grid_min::Float64
    grid_max::Float64
end

function (g::GenNDfunc)(b)
    nbit = Int(length(b)/g.d) # number of qubits ## 2
    N = 2^nbit # number of data points ## 4
    range_ = g.grid_max - g.grid_min

    z = Array{Float64}(undef, g.d)
    for i in 1:g.d
        ran = (i-1)*nbit+1:i*nbit
        dec_z = frombins(b[ran])
        z[i] = range_ * (dec_z - 1)/N + g.grid_min
    end

    return g.func(z) 
end

In [42]:
countelem(M) = sum([prod(size(x)) for x in M])

countelem (generic function with 1 method)

In [43]:
function tobins(i::Int, nbit::Int)::Vector{Int} # convert integer to binary ## (i=3, nbit=4)  
    @assert 1 ≤ i ≤ 2^nbit # check an argument "i" ## 1 ≤ 3 ≤ 16
    mask = 1 << (nbit-1) # increase (nbit-1) binary digits from left to right to make "mask" nbit binary digits ## mask = 1 << (4-1) = (1000)_[2] = (8)_[10] 
    # @show string(mask, base=2)
    bin = ones(Int, nbit) # set initial binary in 1 start format ## bin = [1, 1, 1, 1] 
    target_bin = i-1 # convert an argument "i" from 1 start to 0 start format ## target_bin = (3-1)_[10] = (2)_[10] = (0010)_[2] 
    for n in 1:nbit
        bin[n] = ( (mask & target_bin) >> (nbit-n) ) + 1 
        mask = mask >> 1 
    end
    return bin ## [1, 1, 2, 1]
end

tobins (generic function with 1 method)

In [44]:
function frombins(bin::Vector{Int})::Int # convert binary(Vector{Int}) to integer ## bin = [1, 1, 2] 
    @assert all(1 .≤ bin .≤ 2) # check if all elements are 1 or 2
    nbit = length(bin) # number of bits ## nbit = 3
    i = 1 # start from 1 (not 0) to make it 1 start format
    tmp = 2^(nbit-1) # maximum value of transformation rule: 2^(nbit-1), 2^(nbit-2), ..., 2^0 ## tmp = 2^2
    for n in eachindex(bin)
        i += tmp * (bin[n]-1) 
        tmp = tmp >> 1 
    end
    return i ## i = 2
end

frombins (generic function with 1 method)

In [45]:
function tci_oneshot(func, d, localdims, firstpivot, tol)
    BLAS.set_num_threads(4)
    #func_q = GenNDfunc(d, func)

    for isearch in 1:100
        p = TCI.optfirstpivot(func, localdims, firstpivot) # search optimal fist pivot
        if abs(func(p)) > abs(func(firstpivot))
            firstpivot = p
        end
    end

    # execute tci2
    qtt, ranks, errors = TCI.crossinterpolate2(
        ComplexF64,
        func, 
        localdims, 
        [firstpivot], 
        tolerance = tol, 
        maxiter = 20, 
        verbosity = 1, 
        loginterval = 1,
        pivotsearch = :full,
        #normalizeerror=false,
    )

    return qtt, errors
end    

tci_oneshot (generic function with 1 method)

In [46]:
function def_nonDiag(d, mtr::Matrix{Float64}, nondiag::Float64)
    for i in 1:d, j in 1:d
        if i != j
            mtr[i, j] = nondiag
        end
    end
    return mtr
end

def_nonDiag (generic function with 1 method)

$$
v(\vec{S}(T)) = \max \Bigl\lbrace \min \lbrace S^1_T, \ldots, S^d_T \rbrace - K, 0 \Bigl\rbrace
$$

In [47]:
function payoff_v(x::Vector{Float64}, K::Float64)
    return maximum([(minimum(exp.(x)) - K), 0.0])
end
;

$$
q(\vec{x}|\vec{x}_0) = \frac{1}{\sqrt{(2 \pi)^d \det \Sigma}} \exp\left(-\frac{1}{2}\left(\vec{x}-\vec{\mu}\right)^T \Sigma^{-1} \left(\vec{x}-\vec{\mu}\right)\right)
$$

$$
\mu_j = x_0^j+r_j T-\frac{1}{2} \sigma_j^2 \Sigma_{j j} T 
$$

In [48]:
function normal_dstrb_q(d::Int, x::Vector{Float64}, x0::Vector{Float64}, σ::Vector{Float64}, Σ::Matrix{Float64}, T::Float64, r::Float64)
    #μ = [x0[i] + r * T - σ[i]^2 * Σ[i,i] * T / 2 for i in 1:d]
    μ = [x0[i] + r * T - Σ[i,i] * T / 2 for i in 1:d]
    mtrx = transpose(x .- μ) * inv(Σ) * (x .- μ)
    
    return exp(-mtrx/2)/sqrt((2*π)^d * det(Σ))
end

normal_dstrb_q (generic function with 1 method)

In [49]:
function func(idxs::Vector{Int}, f::Function, N::Int, cut::Float64, slice::Float64) <: Function
    return f( cut .* ( idxs ) .+ slice )
end

func (generic function with 1 method)

In [50]:
function evaluate_options_(random_combinations, tt_option)
    result_hako = []
    result_time = []
    for i in random_combinations
        #@show i
        time_inner = @elapsed begin
            result = qo._evaluate(tt_option, i)
        end
        push!(result_hako, result)
        push!(result_time, time_inner)
    end
    return result_hako, result_time
end

evaluate_options_ (generic function with 1 method)

In [51]:
function generate_random_combinations_(d, R, num_samples)
    Random.seed!(100)

    possible_values = collect(1:2)
    random_combinations = Vector{Int}[]

    for _ in 1:num_samples
        combination = rand(possible_values, d*R)
        push!(random_combinations, combination)
    end

    return random_combinations
end

generate_random_combinations_ (generic function with 1 method)

In [52]:
d = 3
T = 1.0
r = 0.01
K = 100.0
s0 = fill(100.0, d)
x0 = log.(s0)
σ = fill(0.2, d)

nondiag = 1/3
Σ_ = def_nonDiag(d, Matrix{Float64}(I, d, d), nondiag)
σ_ = Matrix{Float64}(I, d, d) .* σ[1]
Σ = σ_ * Σ_ * σ_

3×3 Matrix{Float64}:
 0.04       0.0133333  0.0133333
 0.0133333  0.04       0.0133333
 0.0133333  0.0133333  0.04

In [53]:
v(x) = payoff_v(x, K)
q(x) = normal_dstrb_q(d, x, x0, σ, Σ, T, r)

q (generic function with 1 method)

In [54]:
R = 8
N = 2^R
@show N
cut = 0.04
slice = 0.0

grid_min = slice
grid_max = N*cut + slice

@show grid_min
@show grid_max

idxs_ = fill(Int(N/2), d)
xs = cut .* ( idxs_ ) .- slice
xsp =  cut .* ( idxs_ .+ 1 ) .- slice
xsm =  cut .* ( idxs_ .- 1 ) .- slice

v_pre(idx) = func(idx, v, N, cut, slice)
q_pre(idx) = func(idx, q, N, cut, slice)

N = 256
grid_min = 0.0
grid_max = 10.24


q_pre (generic function with 1 method)

In [55]:
localdims = fill(2, d*R)
firstpivot = rand(1:2, d*R)

v_bit = GenNDfunc(d, v, grid_min, grid_max)
q_bit = GenNDfunc(d, q, grid_min, grid_max)

v_tci = TCI.ThreadedBatchEvaluator{Float64}(v_bit, localdims)
q_tci = TCI.ThreadedBatchEvaluator{Float64}(q_bit, localdims)

(::TensorCrossInterpolation.ThreadedBatchEvaluator{Float64}) (generic function with 3 methods)

In [56]:
tol_v = 1e-5
tol_q = 1e-5

1.0e-5

In [57]:
tci_time = @elapsed begin
    @time qtt_v, errors_v = tci_oneshot(v_tci, d, localdims, firstpivot, tol_v)
    @time qtt_q, errors_q = tci_oneshot(q_tci, d, localdims, firstpivot, tol_q)
end

iteration = 1, rank = 8, error= 7.929656931082718e-12, maxsamplevalue= 26803.18607429759, nglobalpivot=4
  Rejected 4 global pivots added in the previous iteration, errors are [1.1368683772161603e-13, 5.684341886080802e-14, 4.547473508864641e-13, 4.547473508864641e-13]
iteration = 2, rank = 22, error= 1.148218198034198e-11, maxsamplevalue= 26803.18607429759, nglobalpivot=5
  Rejected 5 global pivots added in the previous iteration, errors are [4.887303582116817e-15, 5.684341886080801e-13, 5.684341886080801e-13, 2.8421709430404007e-13, 3.410605131648481e-13]
iteration = 3, rank = 40, error= 1.8998169465610582e-11, maxsamplevalue= 26803.18607429759, nglobalpivot=4
  Rejected 4 global pivots added in the previous iteration, errors are [4.547473508864641e-13, 3.410605131648481e-13, 5.684341886080802e-14, 1.8189894035458565e-12]
iteration = 4, rank = 52, error= 1.463359815898904e-11, maxsamplevalue= 26803.18607429759, nglobalpivot=5
  Rejected 5 global pivots added in the previous iteration

19.53942505

In [58]:
M1 = qtt_v.sitetensors
M2 = qtt_q.sitetensors

24-element Vector{Array{ComplexF64, 3}}:
 [1.0 + 0.0im 0.0 + 0.0im;;; 0.0 + 0.0im 1.0 + 0.0im]
 [5.255152110627355e-44 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im -2.7831394338699136e-100 + 0.0im;;; -6.080314439634118e-72 + 0.0im 0.0 + 0.0im; 1.0 + 0.0im 3.5554105278784314e-58 + 0.0im]
 [0.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im -3.7043316045281324e-29 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im; 1.0 + 0.0im 8.885176602250863e-19 - 0.0im;;; 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 2.302937119536514e-28 + 0.0im]
 [0.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im -1.321244736137673e-9 + 0.0im; 1.4062749559831597e-15 - 0.0im 0.0 + 0.0im;;; 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 3.418120450280581e-11 - 0.0im; -8.902921704108317e-13 + 0.0im 0.0 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im; 1.0 + 0.0im 4.5303478801452485e-7 + 0.0im; -1.4299738536930195e-14 + 0.0im 0.0 + 0.0im;;; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -3.1494123689493204e-8 + 0.0im; 4.999737925348586e-9 + 0.0im 1.0 + 0.0im]
 [1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; 0.0 

In [59]:
size.(M1)

24-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 2, 2)
 (2, 2, 3)
 (3, 2, 5)
 (5, 2, 9)
 (9, 2, 18)
 (18, 2, 35)
 (35, 2, 68)
 (68, 2, 83)
 (83, 2, 86)
 (86, 2, 112)
 ⋮
 (124, 2, 78)
 (78, 2, 69)
 (69, 2, 45)
 (45, 2, 28)
 (28, 2, 15)
 (15, 2, 8)
 (8, 2, 4)
 (4, 2, 2)
 (2, 2, 1)

In [60]:
size.(M2)

24-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 2, 2)
 (2, 2, 2)
 (2, 2, 3)
 (3, 2, 4)
 (4, 2, 7)
 (7, 2, 10)
 (10, 2, 9)
 (9, 2, 8)
 (8, 2, 13)
 (13, 2, 13)
 ⋮
 (16, 2, 8)
 (8, 2, 11)
 (11, 2, 11)
 (11, 2, 12)
 (12, 2, 10)
 (10, 2, 7)
 (7, 2, 4)
 (4, 2, 2)
 (2, 2, 1)

In [61]:
smpl_indices_v = []
smpl_indices_q = []
for i in 1:2^7
    x = 2^(R-7) * i
    @show x
    x1s = tobins(x, R)
    x2s_v = tobins(trunc(Int, N*0.8), R)
    x2s_q = tobins(trunc(Int, N*0.6), R)
    push!(smpl_indices_v, vcat(x1s, x2s_v))
    push!(smpl_indices_q, vcat(x1s, x2s_q))
end
@show smpl_indices_v
@show len = length(smpl_indices_q)
;

x = 2
x = 4
x = 6
x = 8
x = 10
x = 12
x = 14
x = 16
x = 18
x = 20
x = 22
x = 24
x = 26
x = 28
x = 30
x = 32
x = 34
x = 36
x = 38
x = 40
x = 42
x = 44
x = 46
x = 48
x = 50
x = 52
x = 54
x = 56
x = 58
x = 60
x = 62
x = 64
x = 66
x = 68
x = 70
x = 72
x = 74
x = 76
x = 78
x = 80
x = 82
x = 84
x = 86
x = 88
x = 90
x = 92
x = 94
x = 96
x = 98
x = 100
x = 102
x = 104
x = 106
x = 108
x = 110
x = 112
x = 114
x = 116
x = 118
x = 120
x = 122
x = 124
x = 126
x = 128
x = 130
x = 132
x = 134
x = 136
x = 138
x = 140
x = 142
x = 144
x = 146
x = 148
x = 150
x = 152
x = 154
x = 156
x = 158
x = 160
x = 162
x = 164
x = 166
x = 168
x = 170
x = 172
x = 174
x = 176
x = 178
x = 180
x = 182
x = 184
x = 186
x = 188
x = 190
x = 192
x = 194
x = 196
x = 198
x = 200
x = 202
x = 204
x = 206
x = 208
x = 210
x = 212
x = 214
x = 216
x = 218
x = 220
x = 222
x = 224
x = 226
x = 228
x = 230
x = 232
x = 234
x = 236
x = 238
x = 240
x = 242
x = 244
x = 246
x = 248
x = 250
x = 252
x = 254
x = 256
smpl_indices_v = Any[[1, 1, 1

In [62]:
xs_p = collect(1:2^(R-7):N);


In [63]:
plot_v = plot(
    xs_p,
    real.(v_tci.(smpl_indices_v))
)

plot!(
    plot_v,
    xs_p,
    real.(evaluate_options_(smpl_indices_v, M1)[1])
)

InexactError: InexactError: Int64(5.333333333333333)

In [64]:
plot(
    xs_p,
    real.(v_tci.(smpl_indices_v)) .- real.(evaluate_options_(smpl_indices_v, M1)[1])
)

UndefVarError: UndefVarError: `qo` not defined

In [65]:
plot_q = plot(
    #yscale = :log10,
    xs_p,
    real.(q_tci.(smpl_indices_q))
)

plot!(
    plot_q,
    xs_p,
    real.(evaluate_options_(smpl_indices_q, M2)[1])
)

InexactError: InexactError: Int64(5.333333333333333)

In [66]:
plot(
    xs_p,
    real.(q_tci.(smpl_indices_q)) .-  real.(evaluate_options_(smpl_indices_q, M2)[1])
)

UndefVarError: UndefVarError: `qo` not defined

In [67]:
num_samples = 100
random_combinations = generate_random_combinations_(d, R, num_samples)
;

In [68]:
max_v = maximum(real.(v_tci.(random_combinations)) .- real.(evaluate_options_(random_combinations, M1)[1]))
max_q = maximum(real.(q_tci.(random_combinations)) .- real.(evaluate_options_(random_combinations, M2)[1]))
@show max_v 
@show max_q 

UndefVarError: UndefVarError: `qo` not defined

$$
V(\vec{p})=\mathbb{E}\left[e^{-rT}v(\vec{S}(T))\middle|\vec{S}_0\right]=e^{-rT} \int_{-\infty}^{\infty} v(\exp(\vec{x}))q(\vec{x}|\vec{x}_0)dx
$$

In [69]:
function inner_product_func(M1::Vector{Array{ComplexF64,3}}, M2::Vector{Array{ComplexF64,3}})
    N = length(M1)  # コアの数
    C = ones(ComplexF64, 1)  # 初期の縮約結果（スカラー）

    for i in 1:N
        G1 = M1[i]         # M1のi番目のコア（サイズ：r1_left × d_i × r1_right）
        G2 = M2[i]         # M2のi番目のコア（サイズ：r2_left × d_i × r2_right）

        # 各コアの次元を取得
        r1_left, d_i, r1_right = size(G1)
        r2_left, _, r2_right = size(G2)

        # 縮約結果Cをベクトルに変形
        C = reshape(C, r1_left * r2_left)

        # 転送行列Tを初期化
        T = zeros(ComplexF64, r1_right * r2_right, r1_left * r2_left)

        # 物理次元（d_i）に沿って縮約
        for e in 1:d_i
            # コアのスライスを取得
            G1_slice = G1[:, e, :]    # サイズ：r1_left × r1_right
            G2_slice = G2[:, e, :]  # サイズ：r2_left × r2_right

            # Kronecker積を計算
            K = kron(G1_slice, G2_slice)  # サイズ：(r1_left * r2_left) × (r1_right * r2_right)

            # 転送行列に追加
            T += K'
        end

        # 縮約結果を更新
        C = T * C
    end

    # 最終的な内積を返す
    return C[1]
end


inner_product_func (generic function with 1 method)

In [70]:
inner_product = inner_product_func(M1, M2)

28340.311044232963 + 0.0im

In [71]:
res = inner_product * exp(- r * T) * (cut^d)

1.795732495215524 + 0.0im

In [72]:
ref = 3.344541197233318

3.344541197233318

In [73]:
real(ref - res)

1.548808702017794

In [74]:
0.00029746139081110456

0.00029746139081110456

In [75]:
real(ref - res) / ref

0.46308555065759227

In [76]:
bonddims_v = Int[]
bonddims_q = Int[]
for i in 2:length(M1)
    push!(bonddims_v, size(M1[i])[1])
    push!(bonddims_q, size(M2[i])[1])
end
@show bonddims_v
@show bonddims_q
;

bonddims_v = [2, 3, 5, 9, 18, 35, 68, 83, 86, 112, 127, 133, 134, 133, 124, 78, 69, 45, 28, 15, 8, 4, 2]
bonddims_q = [2, 2, 3, 4, 7, 10, 9, 8, 13, 13, 16, 22, 30, 27, 16, 8, 11, 11, 12, 10, 7, 4, 2]
